# K-Nearest Neighbors

In [23]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
from scipy import stats

%matplotlib inline

In [3]:
# read in data using pandas
df_train = pd.read_csv('data/train.csv')

In [22]:
# split (training) data into training and testing sets
def split_data(df, fraction=0.9): 
    # get pixels and labels
    pixels = df.iloc[:,1:785].as_matrix()
    labels = df.iloc[:,0].as_matrix()
    
    # normalize data
    pixels = pixels / 255.0
    #pixels = pixels / np.sum(pixels, axis=1)[:,None]
    
    # split into training and testing
    n = pixels.shape[0]
    pixels_train = pixels[0:int(fraction * n)]
    pixels_test = pixels[int(fraction * n):n]
    labels_train = labels[0:int(fraction * n)]
    labels_test = labels[int(fraction * n):n]
    
    return pixels_train, pixels_test, labels_train, labels_test

## KNN
K-nearest neighbors is a simple algoritm that computes the distance between an unlabeld image and the images that are labelled. The K images with the shortest distance and selected and the most frequently occuring label is assigned to the unlabelled image. K-NN does not require any prior training on the data and instead all the computation is done during classification. 

I compute the distance using both the Euclidean (L2) norm and the L3 norm (which got better results according the MNIST data page). 

In [41]:
# knn function
def KNN(pixels_labeled, pixels_unlabeled, labels, K, norm):
    
    n = len(pixels_unlabeled)
    predictions = np.zeros(n)
    
    # loop through each item in the unlabelled dataset
    for i in range(0, n):
        pixel = pixels_unlabeled[i]
        
        # compute distance to labelled data
        dist = np.power( np.sum(np.power((pixel - pixels_labeled), norm), axis=1), 1.0/norm )
        
        # select K nearest neighbors
        indices = np.argsort(dist)
        top_labels = labels[indices[0:K]]
    
        # what is majority label for these elements?
        predictions[i] = stats.mode(top_labels).mode
        
        if (i % 1000 == 0): 
            print('i = ', i)

    return predictions

In [32]:
k = 3

X_train, X_test, y_train, y_test = split_data(df_train, fraction = 0.8)

pred = KNN(X_train, X_test, y_train, k, 2)

In [35]:
percent_error = np.sum(pred != y_test) / len(y_test) * 100
print('Percent error using L2 norm', percent_error)

Percent error using L2 norm 3.04761904762


In [42]:
# Now we try L3 norm 
pred = KNN(X_train, X_test, y_train, k, 3.0)
percent_error = np.sum(pred != y_test) / len(y_test) * 100
print('Percent error using L3 norm', percent_error)

i =  0
i =  1000
i =  2000
i =  3000
i =  4000
i =  5000
i =  6000
i =  7000
i =  8000
Percent error using L3 norm 80.869047619


In [45]:
# Looks like L3 norm performed terribly -- did not look at paper that was referred to, I must be missing something
# We will use L2 norm one to generate the final classifications on the test data
X_train, X_null, y_train, y_null = split_data(df_train, fraction = 1.0)
df_test = pd.read_csv('data/test.csv')
pixels_unlabeled = df_test.as_matrix()
final_pred = KNN(X_train, pixels_unlabeled, y_train, k, 2.0)

i =  0
i =  1000
i =  2000
i =  3000
i =  4000
i =  5000
i =  6000
i =  7000
i =  8000
i =  9000
i =  10000
i =  11000
i =  12000
i =  13000
i =  14000
i =  15000
i =  16000
i =  17000
i =  18000
i =  19000
i =  20000
i =  21000
i =  22000
i =  23000
i =  24000
i =  25000
i =  26000
i =  27000


In [48]:
# save to csv
df_pred = pd.DataFrame()
df_pred['ImageId'] = range(1,28001)
df_pred['Label'] = final_pred.astype(int)
df_pred.to_csv('KNN.csv', index=False)